In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Please run this code with the kernel reinvent.v3.2

In [28]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

### First, we need to install the custom reinvent scoring package to support the Bradley-Terry model

In [29]:
# print python version
import sys

# Print Python version
print(f"Python version: {sys.version}")

Python version: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]


In [30]:
! pip show reinvent_scoring

Name: reinvent-scoring
Version: 0.0.73
Summary: Scoring functions for Reinvent
Home-page: https://github.com/MolecularAI/reinvent-scoring.git
Author: MolecularAI
Author-email: patronov@gmail.com
License: 
Location: /home/springnuance/reinvent-hitl/reinvent-scoring
Editable project location: /home/springnuance/reinvent-hitl/reinvent-scoring
Requires: 
Required-by: 


In [31]:
# ! conda install scikit-learn=0.21.3

##### If there already exists reinvent_scoring, we should uninstall it

In [32]:
! pip uninstall -y reinvent_scoring

Found existing installation: reinvent-scoring 0.0.73
Uninstalling reinvent-scoring-0.0.73:
  Successfully uninstalled reinvent-scoring-0.0.73


##### Now we install the custom reinvent scoring package
##### The flag -e means that the package is installed in editable mode, so that changes to the code will be immediately available without reinstalling the package. All package info is stored in the setup.py file.

In [33]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-chemistry"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-models"

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reinvent-scoring

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-chemistry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-chemistry
    Found existing installation: reinvent-chemistry 0.0.51
    Uninstalling reinvent-chemistry-0.0.51:
      Successfully uninstalled reinvent-chemistry-0.0.51
  Running setup.py develop for reinvent-chemistry

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-models
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-models
    Found existing installation: reinvent-models 0.0.15rc1
    Uninstalling reinvent-models-0.0.15rc1:
      Suc

In [34]:
# ! conda install -y scikit-learn=0.21.3
! pip list | grep reinvent_scoring


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [35]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from scripts.helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.predict import predict_proba_from_model

def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
def run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # path to the base training dataset
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each iteration
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
        learning_rate, # learning rate for training the model in each HITL iteration
        ):

    np.random.seed(seed)
    rng = default_rng(seed)
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = output_dir
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    for REINVENT_round in range(1, num_rounds + 1):
        check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")
        for HITL_iteration in range(1, num_iters + 1):
            check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}")
    
    # multi-parameter optimization (MPO) loop
    print(f"\nRunning DRD2 (one objective) with rounds {num_rounds}, iters {num_iters}, queries {num_queries}, seed {seed}")
    print(f"Results will be saved at {output_dir}")
    
    base_training_dataset_outputs = load_drd2_dataset(feedback_type=feedback_type, 
                                                 data_path=base_training_dataset_path)

    
    print("Loading initial training datasets successfully")

    # ########################### REINVENT rounds ######################################

    for REINVENT_round in range(1, num_rounds + 1):

        print("=====================================")
        print(f"REINVENT round = {REINVENT_round}")
        
        configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                                REINVENT_round_output_dir=f"{output_dir}/REINVENT_round_{REINVENT_round}", 
                                conf_filename=conf_filename)

        print(f"Creating config file: {configuration_JSON_path}.")

        configuration = json.load(open(f"{configuration_JSON_path}"))

        if REINVENT_round == 1:
            current_model_path = model_pretrained_path
        else:
            # The last iteration's model from the previous round
            current_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{num_iters}/{model_pretrained_name}"

        configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

        # write the updated configuration file 

        with open(configuration_JSON_path, 'w') as f:
            json.dump(configuration, f, indent=4, sort_keys=True)
    
        print("Run REINVENT")                
        command = f"{reinvent_env}/bin/python"
        script = f"{reinvent_dir}/input.py"
        stderr_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.err"
        stdout_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.out"

        # Construct the full command to run
        cmd = [command, script, configuration_JSON_path]
        # Open the file to which you want to redirect stderr and stdout
        with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
            # Execute the command
            result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
        # Check the result
        print("Exit code:", result.returncode)
        
        #############################################################################
        # REINVENT HAS OUTPUT THE RESULT in path f"{output_dir}/REINVENT_round_{REINVENT_round}/results" #
        #############################################################################
        
        # Get the high scoring molecules
        output_high_score = read_scaffold_result(f"{output_dir}/REINVENT_round_{REINVENT_round}/results/scaffold_memory.csv", 
                                                 choose_top_smiles=choose_top_smiles)
        
        scaffold_df = output_high_score["scaffold_df"]
        smiles = output_high_score["smiles"]
        
        # store molecule indexes selected for feedback
        selected_feedback = np.empty(0).astype(int)

        ########################### HITL_iteration in each REINVENT round #####################
        
        for HITL_iteration in range(1, num_iters + 1): # T number of HITL_iterations

            print("----------------------------------")
            print(f"HITL iteration = {HITL_iteration}")
            
            # Loading feedback model
            if REINVENT_round == 1 and HITL_iteration == 1:
                feedback_model_path = model_pretrained_path
            elif REINVENT_round != 1 and HITL_iteration == 1:
                # The last iteration's model from the previous round
                feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            else:
                # The previous HITL iteration's model from current round
                feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration - 1}/{model_pretrained_name}"
            
            feedback_model = load_feedback_model(feedback_type, feedback_model_path)
            
            # print(f"The predicted scores are")
            # predicted_scores = predict_proba_from_model(feedback_type, feedback_model, smiles)
            # print(predicted_scores)

            ######################################################## 
            # Select queries number of smiles with Active Learning #
            ########################################################

            if len(smiles) > num_queries:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, num_queries, list(smiles), 
                                                  selected_feedback, acquisition, rng) 
            else:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, len(smiles), list(smiles), 
                                                  selected_feedback, acquisition, rng)
            
            #print(f"Feedback idx at HITL iteration {HITL_iteration}: {new_queried_smiles_indices}")
            
            new_queried_smiles = [smiles[i] for i in new_queried_smiles_indices]
  
            selected_feedback = np.hstack((selected_feedback, new_queried_smiles_indices))

            new_queried_smiles_human_score = smiles_human_score(new_queried_smiles, sigma_noise)
            
            print(f"Human score at HITL iteration {HITL_iteration}: {new_queried_smiles_human_score}")
            
            # use the augmented training data to retrain the model
            new_queried_fps = np.array([compute_fingerprints(smiles) for smiles in new_queried_smiles])

            iteration_training_dataset_outputs = create_drd2_dataset(feedback_type, 
                                                                      new_queried_smiles, 
                                                                      new_queried_smiles_human_score,
                                                                      new_queried_fps)
            
            if feedback_type == "scoring":
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles'])}")
            else:
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles_1'])}")
            
            # combining the base training dataset with the new queried dataset
            base_training_dataset_outputs = combine_drd2_dataset(feedback_type, base_training_dataset_outputs, 
                                                                   iteration_training_dataset_outputs)
            
            if feedback_type == "scoring":
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles'])}")
            else:
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles_1'])}")
            
            # save augmented training data
            save_drd2_dataset(feedback_type, iteration_training_dataset_outputs, f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_queried_data.csv")
            save_drd2_dataset(feedback_type, base_training_dataset_outputs, f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            print(f"Saved augmented training data at {output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            # Retraining the feedback model using the augmented train set
            retrained_feedback_model = retrain_feedback_model(feedback_type, feedback_model, 
                                                              base_training_dataset_outputs,
                                                              training_epochs, learning_rate)
            
            # save the retrained feedback model
            feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/{model_pretrained_name}"
            torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)
  
            print(f"Saved retrained feedback model at {feedback_model_saving_path}")

def run_REINVENT_round(REINVENT_round,
        reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        current_model_path, # Path to the pretrained model 
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
    ):
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = output_dir
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")

    print("=====================================")
    print(f"REINVENT round = {REINVENT_round}")

    configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                            REINVENT_round_output_dir=f"{output_dir}/REINVENT_round_{REINVENT_round}", 
                            conf_filename=conf_filename)

    print(f"Creating config file: {configuration_JSON_path}.")

    configuration = json.load(open(f"{configuration_JSON_path}"))

    configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

    # write the updated configuration file 

    with open(configuration_JSON_path, 'w') as f:
        json.dump(configuration, f, indent=4, sort_keys=True)

    print("Run REINVENT")                
    command = f"{reinvent_env}/bin/python"
    script = f"{reinvent_dir}/input.py"
    stderr_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.err"
    stdout_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.out"

    # Construct the full command to run
    cmd = [command, script, configuration_JSON_path]
    # Open the file to which you want to redirect stderr and stdout
    with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
        # Execute the command
        result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
    # Check the result
    print("Exit code:", result.returncode)

In [36]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


### Running score regression model

In [40]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from scripts.helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.predict import predict_proba_from_model

seed = 42

# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "scoring" # scoring, comparing, ranking

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
model_pretrained_name = "score_regression_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps
batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 56 # number of molecules, pairs or a set of molecules, depending on the task, 
                 # shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 64 * num_iters * 2 # number of top molecules to choose from scaffold. 

training_epochs = 25 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration


for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        print("\n*******************************************************************")
        print(f"Running HITL with {acquisition} acquisition and noise {sigma_noise}")
        output_dir = f"output_score_regression_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

        if os.path.exists(output_dir):
            pass
        else:
            run_HITL_classify(
                    seed, reinvent_dir, reinvent_env, output_dir,
                    feedback_type, # scoring, comparing, ranking
                    base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
                    model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
                    model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                    num_rounds, # number of rounds, corresponding to R in the paper
                    num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
                    num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
                    REINVENT_n_steps, # number of REINVENT optimization steps
                    batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
                    acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
                    sigma_noise, # noise level for simulated chemist's responses
                    choose_top_smiles, # number of top scoring molecules to choose for feedback
                    training_epochs, # number of epochs for training the model in each HITL iteration
                    learning_rate, # learning rate for training the model in each HITL iteration
            )
            
        if not os.path.exists(f"{output_dir}/REINVENT_round_{num_rounds+1}"):
            REINVENT_n_steps_last = 200
            batch_size_last = 64
            # Run the last REINVENT round using the last feedback model
            current_model_path = f"{output_dir}/REINVENT_round_{num_rounds}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            run_REINVENT_round(num_rounds+1,
                reinvent_dir, reinvent_env, output_dir, 
                feedback_type, # scoring, comparing, ranking
                current_model_path, # Path to the pretrained model 
                REINVENT_n_steps_last, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
                batch_size_last, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
            )
        else:
            print(f"Already ran the last REINVENT round for {output_dir}")


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

*******************************************************************
Running HITL with random acquisition and noise 0.0
Already ran the last REINVENT round for output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.0

*******************************************************************
Running HITL with random acquisition and noise 0.1

Running DRD2 (one objective) with rounds 3, iters 5, queries 120, seed 42
Results will be saved at output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1
Loading initial traini

Found local copy...


Loading Score Regression model from /home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth
Human score at HITL iteration 1: [0.05525568222400341, 0.0, 0.06900238534714223, 0.15318277713062292, 0.0, 0.0, 0.15812625898698537, 0.07831979276888852, 0.0, 0.08213649862421882, 0.0, 0.0, 0.4981830164891991, 0.0, 0.0, 0.0, 0.0, 0.04718224444392781, 0.0, 0.0, 0.15123227928153749, 0.06759762817913847, 0.007254748480326786, 0.0, 0.0, 0.01151230454408184, 0.0, 0.0532613258012906, 0.0, 0.17004393677900181, 0.0, 0.18590007304895761, 0.004011367273918292, 0.0, 0.08269776761188075, 0.0, 0.030456271001043922, 0.0, 0.0, 0.0200828190904182, 0.07725809727771432, 0.04354867599360521, 0.008532550901735908, 0.9077631252802459, 0.0, 0.0, 0.0, 0.10601578966066094, 0.04076567582129282, 0.0, 0.050191126323732155, 0.0, 0.3353390305533548, 0.08321061799463798, 0.10937723246383356, 0.10040546779186318, 0.0, 0.0, 0.03790966831509875, 0.11145382910340802, 0.0, 0.0, 0

Found local copy...


Epoch 60, Loss: 0.05723151657730341
Saved retrained feedback model at output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1/REINVENT_round_1/HITL_iteration_1/score_regression_model.pth
----------------------------------
HITL iteration = 2
Loading Score Regression model from output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1/REINVENT_round_1/HITL_iteration_1/score_regression_model.pth
Human score at HITL iteration 2: [0.07962698942421202, 0.0, 0.14097107802181771, 0.0, 0.07877208997481432, 0.22489505730203568, 0.0, 0.0, 0.012089377970846244, 0.0, 0.0, 0.01092337917270048, 0.0, 0.05126778888405369, 0.0, 0.15512947597113502, 0.0, 0.0, 0.08398110132492609, 0.0, 0.025184580984654148, 0.13140161903545475, 0.0, 0.024073736568885697, 0.028110125070014296, 0.0803897497299673, 0.0, 0.0, 0.05231108375454661, 0.04996730496491644, 0.07719604567561356, 0.04911983648559619, 0.0, 0.030037256171663446, 0.047051673376733404, 0.0, 0.18699292758326366, 0.050422454929728557, 

...

Found local copy...


Epoch 60, Loss: 0.8931858399882913
Saved retrained feedback model at output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1/REINVENT_round_2/HITL_iteration_3/score_regression_model.pth
----------------------------------
HITL iteration = 4
Loading Score Regression model from output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1/REINVENT_round_2/HITL_iteration_3/score_regression_model.pth
Human score at HITL iteration 4: [1.0, 0.8377552610483368, 0.020635928650601407, 1.0, 0.5722661842955888, 0.8382976459369984, 0.007970837197046596, 1.0, 0.0, 0.6808529691131744, 1.0, 0.7836072884709874, 1.0, 0.0, 0.554858912730603, 0.0, 0.0, 0.0, 0.8231474498997935, 1.0, 1.0, 0.7878177203101567, 0.0, 0.8746306795645574, 0.9790200605574712, 0.0, 0.16878308935824712, 0.7463758397989737, 0.8058029203949577, 0.026058270384551146, 0.9169996048958062, 0.7863575487225752, 0.9598955177319133, 0.9317937686413134, 0.7441628651778517, 0.19149145848212085, 0.7486327304734743, 1.0, 0.0, 1.

Found local copy...


Epoch 60, Loss: 0.2522689704783261
Saved retrained feedback model at output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1/REINVENT_round_2/HITL_iteration_4/score_regression_model.pth
----------------------------------
HITL iteration = 5
Loading Score Regression model from output_score_regression_n_steps_100/R3_T5_Q120_acq_random_noise_0.1/REINVENT_round_2/HITL_iteration_4/score_regression_model.pth
Human score at HITL iteration 5: [0.9620226643886219, 0.5920332578586874, 0.8643487106512112, 0.6768836396109343, 1.0, 0.9780476005075929, 0.10325254063864495, 0.6813009382575703, 0.7368678156720133, 0.9775100361297172, 0.9869838293959496, 0.7576689731818456, 0.09457684584469819, 0.38124285287833926, 1.0, 0.7516155532029014, 1.0, 0.0, 0.18120404997066925, 1.0, 1.0, 0.6801534585857966, 1.0, 0.0, 0.6108182816737516, 0.0, 0.7984424608421137, 0.0293132115412799, 0.8015277731871245, 0.1940210820712526, 0.8242397881601291, 0.8599972387340555, 0.27948232203282175, 1.0, 0.542469260656

IndexError: index 380 is out of bounds for axis 0 with size 380

### Running Bradley Terry model

In [18]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
  
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "comparing" # scoring, comparing, ranking

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
model_pretrained_name = "bradley_terry_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps
batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 8 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

# The actual queries is actually (num_queries) x (num_queries - 1), since we are comparing each pair of molecules
# As a result, 8 x 7 = 56 pairs of molecules shown to the human

choose_top_smiles = 64 * num_iters * 2 # number of top molecules to choose from scaffold. 

training_epochs = 25 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration

for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        print("\n*******************************************************************")
        print(f"Running HITL with {acquisition} acquisition and noise {sigma_noise}")
        output_dir = f"output_bradley_terry_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

        if os.path.exists(output_dir):
            pass
        else:
            run_HITL_classify(
                    seed, reinvent_dir, reinvent_env, output_dir,
                    feedback_type, # scoring, comparing, ranking
                    base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
                    model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
                    model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                    num_rounds, # number of rounds, corresponding to R in the paper
                    num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
                    num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
                    REINVENT_n_steps, # number of REINVENT optimization steps
                    batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
                    acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
                    sigma_noise, # noise level for simulated chemist's responses
                    choose_top_smiles, # number of top scoring molecules to choose for feedback
                    training_epochs, # number of epochs for training the model in each HITL iteration
                    learning_rate, # learning rate for training the model in each HITL iteration
            )

        if not os.path.exists(f"{output_dir}/REINVENT_round_{num_rounds+1}"):
            REINVENT_n_steps_last = 200
            batch_size_last = 64
            # Run the last REINVENT round using the last feedback model
            current_model_path = f"{output_dir}/REINVENT_round_{num_rounds}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            run_REINVENT_round(num_rounds+1,
                reinvent_dir, reinvent_env, output_dir, 
                feedback_type, # scoring, comparing, ranking
                current_model_path, # Path to the pretrained model 
                REINVENT_n_steps_last, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
                batch_size_last, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
            )
        else:
            print(f"Already ran the last REINVENT round for {output_dir}")


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

*******************************************************************
Running HITL with random acquisition and noise 0.0

Running DRD2 (one objective) with rounds 3, iters 5, queries 8, seed 42
Results will be saved at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0
Loading initial training datasets successfully
REINVENT round = 1
Creating config file: output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  5006
----------------------------------
HITL iteration = 1


Found local copy...


Loading Bradley Terry model successfully from /home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth
Human score at HITL iteration 1: [0.012794059438848513, 0.009591567810738909, 0.00352978573110307, 0.0037539388397401425, 0.002660693177363879, 0.0005912491070823625, 0.006534587595714743, 0.009123164249650971]
New queried dataset size: 56
Combined dataset size: 256
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv
Epoch 5, Loss: 0.1732650939375162
Epoch 10, Loss: 0.17325223796069622
Epoch 15, Loss: 0.1732391994446516
Epoch 20, Loss: 0.17322596907615662


Found local copy...


Epoch 25, Loss: 0.17321249470114708
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/bradley_terry_model.pth
----------------------------------
HITL iteration = 2
Loading Bradley Terry model successfully from output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/bradley_terry_model.pth
Human score at HITL iteration 2: [0.007457904482147725, 0.0006749446241312491, 0.001139318899845858, 0.03846690825807318, 0.004862642395150195, 0.0013335642742527075, 0.001574630645522563, 0.00019475991309651273]
New queried dataset size: 56
Combined dataset size: 312
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_2/iteration_combined_data.csv
Epoch 5, Loss: 0.2886884758869807
Epoch 10, Loss: 0.28866812338431674
Epoch 15, Loss: 0.28864357868830365
Epoch 20, Loss: 0.2886185919245084


...

B32 processed | Human score at HITL iteration 3: [1.0, 1.0, 0.9966590818356338, 0.6770221207109554, 0.8304379331307014, 0.939233569722167, 0.9354426914011453, 0.639583976550153]
New queried dataset size: 56
Combined dataset size: 928
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/iteration_combined_data.csv
Epoch 5, Loss: 0.6136796902865171
Epoch 10, Loss: 0.6131270807236433
Epoch 15, Loss: 0.6126105058938265
Epoch 20, Loss: 0.6121304053813219
Epoch 25, Loss: 0.6116885356605053
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/bradley_terry_model.pth
----------------------------------
HITL iteration = 4
Loading Bradley Terry model successfully from output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/bradley_terry_model.pth
B1 processed | B2 processed | B3 processed | B4 processed 

Found local copy...


B32 processed | Human score at HITL iteration 4: [0.7981849992404525, 0.7265224814278957, 0.47080323539691493, 0.9694086645752619, 0.2171303560736953, 0.9925529056798945, 0.7886733028111992, 0.5226299704514035]
New queried dataset size: 56
Combined dataset size: 984
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/iteration_combined_data.csv
Epoch 5, Loss: 0.8724180882175764
Epoch 10, Loss: 0.8717378551761309
Epoch 15, Loss: 0.8711856106917063
Epoch 20, Loss: 0.8706139177083969
Epoch 25, Loss: 0.8700982307394346
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/bradley_terry_model.pth
----------------------------------
HITL iteration = 5
Loading Bradley Terry model successfully from output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/bradley_terry_model.pth
B1 processed | B2 process

Found local copy...


B32 processed | Human score at HITL iteration 5: [0.9618755899564847, 1.0, 0.9298568181004594, 0.30974141908886244, 0.2768228821951553, 0.8831190838450398, 1.0, 0.6203974934565578]
New queried dataset size: 56
Combined dataset size: 1040
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_5/iteration_combined_data.csv
Epoch 5, Loss: 1.3904367610812187
Epoch 10, Loss: 1.3896121457219124
Epoch 15, Loss: 1.388481080532074
Epoch 20, Loss: 1.3876683972775936
Epoch 25, Loss: 1.3874942660331726
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_5/bradley_terry_model.pth
REINVENT round = 4
Creating config file: output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_4/config.json.
Run REINVENT
Exit code: 0


### Running Rank ListNet model

In [11]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42

# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "ranking" # scoring, comparing, ranking

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
model_pretrained_name = "rank_listnet_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps

# Be careful, as the matrix size could get very large, based on binom(num_queries, 3)
# batch_size at 80 could potentially crash the memory

batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 8 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

# The actual queries is actually binom(num_queries, 3), since we are ranking each set of 3 molecules
# As a result, binom(8, 3) = 56 sets of molecules are shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 64 * num_iters * 2 # number of top molecules to choose from scaffold. 

training_epochs = 25 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration


for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        print("\n*******************************************************************")
        print(f"Running HITL with {acquisition} acquisition and noise {sigma_noise}")
        
        output_dir = f"output_rank_listnet_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

        if os.path.exists(output_dir):
            pass
        else:
            run_HITL_classify(
                    seed, reinvent_dir, reinvent_env, output_dir,
                    feedback_type, # scoring, comparing, ranking
                    base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
                    model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
                    model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                    num_rounds, # number of rounds, corresponding to R in the paper
                    num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
                    num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
                    REINVENT_n_steps, # number of REINVENT optimization steps
                    batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
                    acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
                    sigma_noise, # noise level for simulated chemist's responses
                    choose_top_smiles, # number of top scoring molecules to choose for feedback
                    training_epochs, # number of epochs for training the model in each HITL iteration
                    learning_rate, # learning rate for training the model in each HITL iteration
            )
        
        if not os.path.exists(f"{output_dir}/REINVENT_round_{num_rounds+1}"):
            REINVENT_n_steps_last = 200
            batch_size_last = 64
            # Run the last REINVENT round using the last feedback model
            current_model_path = f"{output_dir}/REINVENT_round_{num_rounds}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            run_REINVENT_round(num_rounds+1,
                reinvent_dir, reinvent_env, output_dir, 
                feedback_type, # scoring, comparing, ranking
                current_model_path, # Path to the pretrained model 
                REINVENT_n_steps_last, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
                batch_size_last, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
            )
        else:
            print(f"Already ran the last REINVENT round for {output_dir}")

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

*******************************************************************
Running HITL with random acquisition and noise 0.0
Already ran the last REINVENT round for output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0

*******************************************************************
Running HITL with random acquisition and noise 0.1
Already ran the last REINVENT round for output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1

*******************************************************************
Running HITL with uncerta

Found local copy...


B9 processed | Human score at HITL iteration 1: [0.0016792732525271448, 0.001355437265589983, 0.008549279621678502, 0.019175283269487694, 0.001136894957438439, 0.004610941209349772, 0.0004182572864760244, 0.02250796278514377]
New queried dataset size: 56
Combined dataset size: 256
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_uncertainty_noise_0.0/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv
Epoch 5, Loss: 0.013360511627979577
Epoch 10, Loss: 0.013342257007025182
Epoch 15, Loss: 0.013323852559551597
Epoch 20, Loss: 0.013304785592481494
Epoch 25, Loss: 0.013285338878631592
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_uncertainty_noise_0.0/REINVENT_round_1/HITL_iteration_1/rank_listnet_model.pth
----------------------------------
HITL iteration = 2
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_uncertainty_noise_0.0/REINVENT_round_1/HITL_iteration_1/rank_listnet_m

Found local copy...


B9 processed | Human score at HITL iteration 2: [0.04755637579597877, 0.000455267920529446, 0.020729585542425454, 0.00013574186965920984, 2.8883748421837482e-05, 0.006243266774914877, 0.057159920977922114, 0.0014024616306254729]
New queried dataset size: 56
Combined dataset size: 312
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_uncertainty_noise_0.0/REINVENT_round_1/HITL_iteration_2/iteration_combined_data.csv
Epoch 5, Loss: 0.018030582927167416
Epoch 10, Loss: 0.018124034628272057
Epoch 15, Loss: 0.018069103204955656
Epoch 20, Loss: 0.018038115231320262
Epoch 25, Loss: 0.01809461632122596
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_uncertainty_noise_0.0/REINVENT_round_1/HITL_iteration_2/rank_listnet_model.pth
----------------------------------
HITL iteration = 3
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_uncertainty_noise_0.0/REINVENT_round_1/HITL_iteration_2/rank_listnet

...

B9 processed | Human score at HITL iteration 3: [0.37626816426938925, 0.18004263275384036, 0.7188419719732789, 0.5001598747383915, 0.2589451819928762, 0.14375624616334454, 0.9415088504237032, 0.6478535735997335]
New queried dataset size: 56
Combined dataset size: 928
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/iteration_combined_data.csv
Epoch 5, Loss: 0.006952215146156959
Epoch 10, Loss: 0.006118910576333292
Epoch 15, Loss: 0.005362768715713173
Epoch 20, Loss: 0.0046865976037224755
Epoch 25, Loss: 0.004091843540663831
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/rank_listnet_model.pth
----------------------------------
HITL iteration = 4
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/rank_listnet_model.pth
B1 processed | B2 p

Found local copy...


B9 processed | Human score at HITL iteration 4: [0.03141386041497231, 0.17347257136416272, 1.0, 0.046187420581074744, 0.09908726865990543, 0.8666034750160369, 0.8019295246587013, 0.7070610750375306]
New queried dataset size: 56
Combined dataset size: 984
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/iteration_combined_data.csv
Epoch 5, Loss: 0.007375501814143111
Epoch 10, Loss: 0.006218636796499292
Epoch 15, Loss: 0.005146526993485168
Epoch 20, Loss: 0.0043102421623189
Epoch 25, Loss: 0.0036158408717407533
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/rank_listnet_model.pth
----------------------------------
HITL iteration = 5
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/rank_listnet_model.pth
B1 processed | B2 processed | B3 p

Found local copy...


B9 processed | Human score at HITL iteration 5: [0.5422065558084836, 1.0, 0.3547599697268366, 0.5947294151167312, 0.05702801976348898, 0.0, 0.364982407951798, 0.3659699829294827]
New queried dataset size: 56
Combined dataset size: 1040
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_5/iteration_combined_data.csv
Epoch 5, Loss: 0.005981626745779067
Epoch 10, Loss: 0.004779673501616344
Epoch 15, Loss: 0.003866765327984467
Epoch 20, Loss: 0.0032627856417093426
Epoch 25, Loss: 0.0027346378774382174
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_5/rank_listnet_model.pth
REINVENT round = 4
Creating config file: output_rank_listnet_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_4/config.json.
Run REINVENT
Exit code: 0
